In [1]:
# Import Dependencies
import os
import numpy as np
import pandas as pd
import utils as utils
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist

import warnings
warnings.filterwarnings("ignore")

### Preprocessing

In [2]:
# Import and read the data
df_raw = pd.read_csv("data/top_10000_1960-now.csv")

In [3]:
# list columns for features and target
df_raw.columns

Index(['Track URI', 'Track Name', 'Artist URI(s)', 'Artist Name(s)',
       'Album URI', 'Album Name', 'Album Artist URI(s)',
       'Album Artist Name(s)', 'Album Release Date', 'Album Image URL',
       'Disc Number', 'Track Number', 'Track Duration (ms)',
       'Track Preview URL', 'Explicit', 'Popularity', 'ISRC', 'Added By',
       'Added At', 'Artist Genres', 'Danceability', 'Energy', 'Key',
       'Loudness', 'Mode', 'Speechiness', 'Acousticness', 'Instrumentalness',
       'Liveness', 'Valence', 'Tempo', 'Time Signature', 'Album Genres',
       'Label', 'Copyrights'],
      dtype='object')

In [4]:
# Drop unnecessary columns
# Listing all columns for review
# all columns listed, columns to keep are commented out.
df_data = df_raw.drop([#'Track URI',
                       'Track Name',
                       'Artist URI(s)',
                       'Artist Name(s)',
                       'Album URI',
                       'Album Name',
                       'Album Artist URI(s)',
                       'Album Artist Name(s)',
                       'Album Release Date',
                       'Album Image URL',
                       'Disc Number',
                       'Track Number',
                       'Track Duration (ms)',
                       'Track Preview URL',
                       #'Explicit',
                       'Popularity',
                       'ISRC',
                       'Added By',
                       'Added At',
                       'Artist Genres',
                       #'Danceability',
                       #'Energy',
                       #'Key',
                       'Loudness',
                       'Mode',
                       #'Speechiness',
                       #'Acousticness',
                       #'Instrumentalness',
                       #'Liveness',
                       #'Valence',
                       #'Tempo',
                       'Time Signature',
                       'Album Genres',
                       'Label',
                       'Copyrights'],
                       axis=1)

In [5]:
# Renaming columns to convention
df_data = df_data.rename(columns={
                   'Track URI': 'track_uri',
                   'Album Image URL': 'image',
                   'Explicit': 'explicit',
                   'Popularity': 'popularity',
                   'Danceability': 'danceability',
                   'Energy': 'energy',
                   'Key': 'key',
                   'Speechiness': 'speechiness',
                   'Acousticness': 'acousticness',
                   'Instrumentalness': 'instrumentalness',
                   'Liveness': 'liveness',
                   'Valence': 'valence',
                   'Tempo': 'tempo'
       })

In [6]:
# Verifying name update and
# Reviewing datatypes for analysis
df_data.dtypes

track_uri            object
explicit               bool
danceability        float64
energy              float64
key                 float64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
dtype: object

In [7]:
# drop nulls
df_data = df_data.dropna()
# reset index
df_data = df_data.reset_index(drop=True)


In [8]:
# Convert 'explicit' column from boolean to binary
df_data['explicit'] = df_data['explicit'].astype(int)
# verify update
df_data.head()

,track_uri,explicit,danceability,energy,key,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,spotify:track:1XAZlnVtthcDZt2NI1Dtxo,0,0.617,0.872,8.0,0.0480,0.0158,0.112000,0.4080,0.504,111.458
1,spotify:track:6a8GbQIlV8HBUW3c6Uk9PH,0,0.825,0.743,2.0,0.1490,0.0142,0.000021,0.2370,0.800,127.045
2,spotify:track:70XtWbcVZcpaOddJftMcVi,0,0.677,0.665,7.0,0.0305,0.5600,0.000001,0.3380,0.706,74.981
3,spotify:track:1NXUWyPJk5kO6DQJ5t7bDu,0,0.683,0.728,9.0,0.2590,0.5680,0.000051,0.0384,0.833,75.311
4,spotify:track:72WZtWs6V7uu3aMgMmEkYe,0,0.319,0.627,0.0,0.0687,0.6750,0.000073,0.2890,0.497,85.818


KNN Testing

In [12]:
# Ensure all column names are strings
df_data.columns = df_data.columns.astype(str)

In [ ]:
# Extract relevant features
X = df_data.drop(columns='track_uri')

In [ ]:
# Example target labels (binary classification)
# Replace this with your actual target column
y = df_data['track_uri']

In [14]:
# Normalize/Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [15]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

NameError: name 'train_test_split' is not defined

### what other models to test?
- gaussian?
- DBSCAN?
- agglomarative clustering?
